In [2]:
import numpy
import pandas
#import keras

In [3]:
ds_test = pandas.read_csv("/Users/aisha/desktop/AI/we_data/test.csv")
ds_train = pandas.read_csv("/Users/aisha/desktop/AI/we_data/train.csv")
ds_val = pandas.read_csv("/Users/aisha/desktop/AI/we_data/validation.csv")

# Dataset Info

## Training Data

In [4]:
print('headers:', list(ds_train))

print('size:', len(ds_train))
ds_train.head()

headers: ['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag']
size: 2430981


click  weekday  hour                                     bidid  \
0      0        5    22  b7bea80521fdecd95d2d761a38c91c3f09618066   
1      0        1    20  4f51205475678f5a124bc76b2c54163bf8eaa7eb   
2      0        3    13  b604e3fd054a658ab7ced4285ebf2ef54d2bd890   
3      0        6    23  0348beeae93e561584c3b50fc9e7746a33048ad7   
4      0        5     6  268149c1789bce2bc9798ffd97ec431219bafeb3   

                                     userid       useragent             IP  \
0  2e880fb7d690cf7377b2e42e701728e3f3c0e4c1      windows_ie   125.37.175.*   
1  3a1fe01360ff8100e7d006b83b77a3e4c01d928c  windows_chrome    171.36.92.*   
2  801d18a056b6fe6b06a794aef17fb0d6daff2414      windows_ie    59.46.106.*   
3  0d6eaf2259699990e38a1fc5116f112070b9ecdc      windows_ie  114.250.226.*   
4  a239d9bb642460d974ba67f85e63b8d3e214da0e      windows_ie   183.63.192.*   

   region  city  adexchange           ...            slotheight  \
0       2     2         2.0           ...                   200   
1     238   239         1.0           ...                   250   
2      40    41         2.0           ...                   250   
3       1     1         1.0           ...                   600   
4     216   233         2.0           ...                    90   

  slotvisibility  slotformat slotprice                          creative  \
0              2           0         5  a4f763f78ef3eedfe614263b94a8924e   
1     FourthView          Na         0                             10722   
2              2           0         5  798b2d49952d77f1eace9f23c210d0b5   
3              2           1         0  cb7c76e7784031272e37af8e7e9b062c   
4      OtherView          Na       133                              7330   

   bidprice payprice                           keypage  advertiser  \
0       238        5  0f951a030abdaedd733ee8d114ce2944        3427   
1       294       23                               NaN        2821   
2       238       24  0f951a030abdaedd733ee8d114ce2944        3427   
3       300       25  bebefa5efe83beee17a3d245e7c5085b        1458   
4       277      133                               NaN        2259   

                   usertag  
0                      NaN  
1                      NaN  
2  10052,10006,13866,10110  
3        13866,10063,10111  
4                      NaN  

[5 rows x 25 columns]

In [5]:
df = ds_train

# Selection of useful fields per advertise
tmp1 = df[['click', 'bidprice', 'payprice', 'advertiser']].groupby('advertiser').sum()
# Calcuate impressions per advertiser
tmp2 = df[['click', 'bidprice', 'payprice', 'advertiser']].groupby('advertiser').size().reset_index(name='impressions')

# Joining the 2 dataframes
stats = tmp1.join(tmp2.set_index('advertiser'), on='advertiser')

# Calculate Click-Through Rate (CTR)
# https://en.wikipedia.org/wiki/Click-through_rate
stats['ctr'] = stats['click'] / stats['impressions'] * 100

# Calculate cost-per-mille (CPM) 
# - cost advertisers pay for 1k views or clicks
# https://en.wikipedia.org/wiki/Cost_per_mille
stats['cpm'] = stats['payprice'] / stats['impressions']

# Calculate effective cost-per-click (eCPC) 
# Cost to advertiser / number of clicks
stats['ecpc'] = stats['payprice'] / stats['click']

stats

click   bidprice  payprice  impressions       ctr        cpm  \
advertiser                                                                 
1458          385  147705900  33968736       492353  0.078196  68.992646   
2259           43   38508376  12428238       133673  0.032168  92.974931   
2261           36   31701699   9873779       110122  0.032691  89.662184   
2821          131   61343675  18828044       211366  0.061978  89.077922   
2997          217   13802633   3129267        49829  0.435489  62.800116   
3358          202   61729738  22447231       264956  0.076239  84.720599   
3386          320  136512300  34931823       455041  0.070323  76.766320   
3427          272   95218860  30458711       402806  0.067526  75.616329   
3476          187   77042367  23918779       310835  0.060161  76.950083   

                     ecpc  
advertiser                 
1458         88230.483117  
2259        289028.790698  
2261        274271.638889  
2821        143725.526718  
2997         14420.585253  
3358        111124.905941  
3386        109161.946875  
3427        111980.555147  
3476        127907.909091

In [6]:
uniq_fields = [ x for x in (ds_train) if x not in list(ds_test)]

data = ds_val
print('Fields in training/validation, not in test:', uniq_fields)

print(f'''---Mean---
{data[uniq_fields].mean()}
---Max---
{data[uniq_fields].max()}
---Min---
{data[uniq_fields].min()}
''')

Fields in training/validation, not in test: ['click', 'bidprice', 'payprice']
---Mean---
click         0.000665
bidprice    273.053099
payprice     78.234005
dtype: float64
---Max---
click         1
bidprice    300
payprice    300
dtype: int64
---Min---
click         0
bidprice    227
payprice      0
dtype: int64



## Validation Data

In [7]:
print('headers:', list(ds_val))
print('size:', len(ds_val))
ds_val.head()

headers: ['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag']
size: 303925


click  weekday  hour                                     bidid  \
0      0        4    20  bbcb813b6166538503d8b33a5602d7d72f6019dc   
1      0        1    21  5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d   
2      0        4     8  f6ece71dae81d6b16bfb24ad6dd5611472d4c673   
3      0        5    15  b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb   
4      0        1    18  0899bf144249458ea9c89188473694bf44c7ca15   

                                     userid       useragent             IP  \
0  663169f66491c98c69f1f94a5c48fa34aa9fe06f      windows_ie  211.144.203.*   
1  a23f0cfab6592c137f796e68fa752fceba08bb1b  windows_chrome   58.247.250.*   
2  5105a013ea54a9706146033e6a138d5234a3803e      windows_ie   117.12.111.*   
3  577e9a768ce17e73c61ba7022db927f9ee761830  windows_chrome   61.187.224.*   
4  e4bf79c562745d671b19f2edf7fda89c2e25987f  windows_chrome   117.41.145.*   

   region  city  adexchange                       ...                         \
0      79    79         1.0                       ...                          
1      79    79         1.0                       ...                          
2       2     2         2.0                       ...                          
3     201   205         2.0                       ...                          
4     134   135         2.0                       ...                          

  slotheight slotvisibility  slotformat slotprice  \
0        600              2           1         0   
1         90              0           1         0   
2        250              2           0         5   
3        280              2           0         5   
4        200              1           0         5   

                           creative  bidprice payprice  \
0  cb7c76e7784031272e37af8e7e9b062c       300       23   
1  c938195f9e404b4f38c7e71bf50263e5       238       75   
2  911b2d84826786018761e8c0b0a3a60c       238       65   
3  dc0998c10f8f0b623b5d949e8272e4c7       238        6   
4  23485fcd23122d755d38f8c89d46ca56       249        5   

                            keypage  advertiser  \
0  bebefa5efe83beee17a3d245e7c5085b        1458   
1  a3becb967c015e9acb5e72f22b100ef8        3476   
2  d29e59bf0f7f8243858b8183f14d4412        3358   
3  d29e59bf0f7f8243858b8183f14d4412        3358   
4  7e0e909ea89f707bc20f9585c9072081        3476   

                                           usertag  
0                                      13866,10111  
1  10075,10057,10024,10052,10083,10063,10006,10110  
2  13866,10024,10059,10063,13776,10083,10006,10111  
3                    13866,10057,10006,10063,10110  
4                          10006,10063,13776,10110  

[5 rows x 25 columns]

In [27]:
df = ds_val

# Selection of useful fields per advertise
tmp1 = df[['click', 'bidprice', 'payprice', 'advertiser']].groupby('advertiser').sum()
# Calcuate impressions per advertiser
tmp2 = df[['click', 'bidprice', 'payprice', 'advertiser']].groupby('advertiser').size().reset_index(name='impressions')

# Joining the 2 dataframes
stats = tmp1.join(tmp2.set_index('advertiser'), on='advertiser')

# Calculate Click-Through Rate (CTR)
# https://en.wikipedia.org/wiki/Click-through_rate
stats['ctr'] = stats['click'] / stats['impressions'] * 100

# Calculate cost-per-mille (CPM) - cost advertisers pay for 1k views or clicks
# https://en.wikipedia.org/wiki/Cost_per_mille
stats['cpm'] = stats['payprice'] / stats['impressions']

# Calculate effective cost-per-click (eCPC) 
# Cost to advertiser / number of clicks
stats['ecpc'] = stats['payprice'] / stats['click']

#######

#calculate constant bidding
stats['avg'] = stats['bidprice']/ stats['impressions']

#ccalculate total bidprice
total1 = stats['bidprice'].sum()
print ('Total bidprice=',total)

#calculate total impression
total2 = stats['impressions'].sum()
print ('Total impressions=',total)

#calculate constant bidding
cons = total1/total2
print('Constant bidding=',cons)



stats

Total bidprice= 82987663
Total impressions= 82987663
Constant bidding= 273.0530986263058


click  bidprice  payprice  impressions       ctr        cpm  \
advertiser                                                                
1458           49  18705900   4294602        62353  0.078585  68.875627   
2259            2   4814539   1568808        16715  0.011965  93.856297   
2261            3   3900349   1214876        13550  0.022140  89.658745   
2821           23   7691157   2394900        26503  0.086783  90.363355   
2997           26   1710752    388784         6176  0.420984  62.950777   
3358           23   7673642   2794021        32939  0.069826  84.824099   
3386           28  16999500   4350793        56665  0.049413  76.780958   
3427           37  11863984   3776735        50183  0.073730  75.259251   
3476           11   9627840   2993751        38841  0.028321  77.077083   

                     ecpc         avg  
advertiser                             
1458         87644.938776  300.000000  
2259        784404.000000  288.037033  
2261        404958.666667  287.848635  
2821        104126.086957  290.199487  
2997         14953.230769  277.000000  
3358        121479.173913  232.965239  
3386        155385.464286  300.000000  
3427        102073.918919  236.414403  
3476        272159.181818  247.878273

## Test Data

In [9]:
print('headers:', list(ds_test))
print('size:', len(ds_test))
ds_test.head()

headers: ['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'keypage', 'advertiser', 'usertag']
size: 303375


weekday  hour                                     bidid  \
0        0    12  366c563de7d90feb9d4dab53e795a93fb3157387   
1        3    14  29167d4caa719788b5a342dbaa25151d53121f80   
2        5    19  ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e   
3        0    21  844c2da00d45315f20b748ec131c26ee99a7cbc7   
4        2    20  c6017f0ad0c44d7d0c9b62583ea863f28941c0ca   

                                     userid       useragent             IP  \
0  75045dd2f2136c93fe55fe6c446ec1527ed8f0bb  windows_chrome    27.197.36.*   
1  11279eb1f8f7a88f877db911673522b6ff202aa7  android_safari  124.126.227.*   
2  4771a819a3f5b86776d8a9456f4f2506578f78d8      windows_ie  116.116.104.*   
3  5360671379f3204afa9cc0edd8fd2e2096cb09eb      windows_ie   115.46.145.*   
4  d791b5f6306b9d299f81daa5448ced7e5bfa010f      windows_ie  221.199.203.*   

   region  city  adexchange                            domain  \
0     146   159         3.0           DDTSQuf0MTTNaqKIvMpENpn   
1       1     1         NaN  d63d96468f66986d5a8eb5225ad0b6f0   
2      27    34         1.0            trqRTJkrBoq7JsNr5SqfNX   
3     238   245         4.0  6d27caf5a202ec085f80af7163ef7cb0   
4      27    35         3.0               3FKElpuEMusyJqKbuKz   

                         ...                          \
0                        ...                           
1                        ...                           
2                        ...                           
3                        ...                           
4                        ...                           

                         slotid  slotwidth slotheight  slotvisibility  \
0             Astro_F_Rectangle        300        250               0   
1                    2745306850        320         50       FirstView   
2  mm_34022157_3445226_11175100        336        280               2   
3                        210842        960         90       FirstView   
4           Fashion_F_Rectangle        300        250               0   

   slotformat slotprice                          creative  \
0           0        10  00fccc64a1ee2809348509b7ac2a97a5   
1          Na         5                             11908   
2           1         0  77819d3e0b3467fe5c7b16d68ad923a1   
3          Na        30                             10717   
4           0        50  44966cc8da1ed40c95d59e863c8c75f0   

                            keypage advertiser  \
0  b2e35064f3549d447edbbdfb1f707c8c       3427   
1                               NaN       2997   
2  bebefa5efe83beee17a3d245e7c5085b       1458   
3                               NaN       2821   
4  43f4a37f42a7c5e6219e2601b26c6976       3386   

                                             usertag  
0  10024,10077,10075,10063,10031,10102,10006,1304...  
1                                                NaN  
2  13866,10006,10024,10059,10048,10063,10067,1008...  
3          10075,13042,10006,10110,13776,10031,10063  
4                                              10063  

[5 rows x 22 columns]